<img src='../img/bimbo.png'>

- Task: Demand = Sales - Returns
- Create holdout test set

## Data exploration

In [83]:
import pandas as pd
import numpy as np
import os
dir_sub = "C:/Users/568377/Cognizant/Data Science/kaggle/Bimbo Bread/submissions/"
dir_raw = "C:/Users/568377/Cognizant/Data Science/kaggle/Bimbo Bread/raw/"

# Feature engineering

https://www.kaggle.com/cast42/rossmann-store-sales/xgboost-in-python-with-rmspe-v2/code

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import operator
import matplotlib

# sub 2

In [85]:
train = pd.read_csv(os.path.join(dir_raw,"sample_submission.csv"))

In [86]:
train["Demanda_uni_equil"]=6

In [87]:
train.to_csv(os.path.join(dir_sub, "submission_2.csv"), index=False)